In [18]:
import torch
import torch.nn as nn

In [19]:
###Lazy way###
# The complete model is saved.
# The serialized data is bound to the specific classes, and the exact directory structure that is used when the model is saved.
##############
# torch.save(arg, PATH)
# torch.load(PATH)
# model.eval()

###recomended###
# Save the parameters
################

# torch.save(model.state_dict(), PATH)

# model must be created again with parameters
# model = Model(*args, **kwargs)
# model.load_state_dict(torch.load(PATH))
# model.eval()

In [34]:
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

device = torch.device("cuda")

model = Model(n_input_features = 6)
model.to(device)

learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

print(model.state_dict())
print(optimizer.state_dict())


OrderedDict([('linear.weight', tensor([[0.0277, 0.0098, 0.3169, 0.0915, 0.3808, 0.2297]], device='cuda:0')), ('linear.bias', tensor([-0.1544], device='cuda:0'))])
{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1]}]}


## Method 1

In [23]:
FILE = "outputs/model.pth"
torch.save(model, FILE)

In [24]:
model = torch.load(FILE)
model.eval()
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.2766, -0.2232,  0.2242, -0.3160, -0.1199,  0.1274]],
       requires_grad=True)
Parameter containing:
tensor([0.1279], requires_grad=True)


## Method 2

In [35]:
FILE = "outputs/model.pth"
torch.save(model.state_dict(), FILE)

In [26]:
loaded_model = Model(n_input_features=6)
loaded_model.load_state_dict(torch.load(FILE))
loaded_model.eval()

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [27]:
for param in model.parameters():
    print(param)
print()
for param in loaded_model.parameters():
    print(param)

Parameter containing:
tensor([[-0.2766, -0.2232,  0.2242, -0.3160, -0.1199,  0.1274]],
       requires_grad=True)
Parameter containing:
tensor([0.1279], requires_grad=True)

Parameter containing:
tensor([[-0.2766, -0.2232,  0.2242, -0.3160, -0.1199,  0.1274]],
       requires_grad=True)
Parameter containing:
tensor([0.1279], requires_grad=True)


## Save during training

In [36]:
checkpoint = {
    "epoch": 90,
    "model_state": model.state_dict(),
    "optim_state": optimizer.state_dict()
}
torch.save(checkpoint, "outputs/checkpoint.pth")

In [39]:
loaded_checkpoint = torch.load("outputs/checkpoint.pth")
epoch = loaded_checkpoint["epoch"]

model = Model(n_input_features=6)
model.load_state_dict(loaded_checkpoint["model_state"])
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0)
optimizer.load_state_dict(loaded_checkpoint["optim_state"])

print(optimizer.state_dict())
print(model.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1]}]}
OrderedDict([('linear.weight', tensor([[0.0277, 0.0098, 0.3169, 0.0915, 0.3808, 0.2297]], device='cuda:0')), ('linear.bias', tensor([-0.1544], device='cuda:0'))])


In [41]:
# Save on CPU, Load on GPU

device = torch.device("cuda")
model = Model(n_input_features=6)
model.load_state_dict(torch.load(FILE, map_location="cuda:0"))
model.to(device)
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[0.0277, 0.0098, 0.3169, 0.0915, 0.3808, 0.2297]], device='cuda:0')), ('linear.bias', tensor([-0.1544], device='cuda:0'))])
